In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")
# This is where you log the name of the experiment and store it in the database

<Experiment: artifact_location='/workspaces/mlops_zoomcamp_test/02-experiment-tracking/mlruns/1', creation_time=1744036646591, experiment_id='1', last_update_time=1744036646591, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred,)**.5

7.758715209663881

In [14]:
# mlflow.sklearn.autolog()

In [ ]:
# lr.fit(X_train, y_train)

2025/04/15 15:13:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd39f1f67542449b8bd5bdd31a51c7cd0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


In [14]:
!mkdir -p models

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "aditya")

    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    alpha = 0.5
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", mse**.5)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)**.5
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 15, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.80022                                                                                                                                                                       
[1]	validation-rmse:8.42692                                                                                                                                                                       
  0%|                                                                                                                                                      | 0/50 [00:00<?, ?trial/s, best loss=?]

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:24:28] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.68533                                                                                                                                                                       
[3]	validation-rmse:7.28931                                                                                                                                                                       
[4]	validation-rmse:7.08155                                                                                                                                                                       
[5]	validation-rmse:6.96739                                                                                                                                                                       
[6]	validation-rmse:6.90982                                                                                                                                                                       
[7]	validation-rmse:6.873

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:25:52] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.88164                                                                                                                                                                      
[1]	validation-rmse:9.84597                                                                                                                                                                       
[2]	validation-rmse:9.05349                                                                                                                                                                       
[3]	validation-rmse:8.45291                                                                                                                                                                       
[4]	validation-rmse:8.00214                                                                                                                                                                       
[5]	validation-rmse:7.665

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:27:35] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.89761                                                                                                                                                                       
[1]	validation-rmse:6.73905                                                                                                                                                                       
[2]	validation-rmse:6.72675                                                                                                                                                                       
[3]	validation-rmse:6.72021                                                                                                                                                                       
[4]	validation-rmse:6.70853                                                                                                                                                                       
[5]	validation-rmse:6.701

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:28:21] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.38284                                                                                                                                                                       
[1]	validation-rmse:7.97928                                                                                                                                                                       
[2]	validation-rmse:7.32633                                                                                                                                                                       
[3]	validation-rmse:7.02708                                                                                                                                                                       
[4]	validation-rmse:6.88918                                                                                                                                                                       
[5]	validation-rmse:6.817

KeyboardInterrupt: 

In [21]:
with mlflow.start_run():
    
    # train = xgb.DMatrix(X_train, label=y_train)
    # valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.1707696537350853,
        'max_depth': 11,
        'min_child_weight': 2.8409933117146555,
        'objective': 'reg:linear',
        'reg_alpha': 0.009894280601251363,
        'reg_lambda': 0.19437455821272276,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)**.5
    mlflow.log_metric("rmse", rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [15:15:23] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:10.88164
[1]	validation-rmse:9.84597
[2]	validation-rmse:9.05349
[3]	validation-rmse:8.45291
[4]	validation-rmse:8.00214
[5]	validation-rmse:7.66514
[6]	validation-rmse:7.41464
[7]	validation-rmse:7.23058
[8]	validation-rmse:7.09371
[9]	validation-rmse:6.99143
[10]	validation-rmse:6.91491
[11]	validation-rmse:6.85745
[12]	validation-rmse:6.81244
[13]	validation-rmse:6.77796
[14]	validation-rmse:6.75139
[15]	validation-rmse:6.73195
[16]	validation-rmse:6.71527
[17]	validation-rmse:6.70173
[18]	validation-rmse:6.69125
[19]	validation-rmse:6.68182
[20]	validation-rmse:6.67488
[21]	validation-rmse:6.66875
[22]	validation-rmse:6.66371
[23]	validation-rmse:6.65980
[24]	validation-rmse:6.65686
[25]	validation-rmse:6.65533
[26]	validation-rmse:6.65119
[27]	validation-rmse:6.64974
[28]	validation-rmse:6.64729
[29]	validation-rmse:6.64558
[30]	validation-rmse:6.64362
[31]	validation-rmse:6.64225
[32]	validation-rmse:6.64109
[33]	validation-rmse:6.63889
[34]	validation-rmse:6.

/usr/local/python/3.12.1/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [15:15:36] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/04/16 15:15:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [22]:
# check that you can make predictions based on loaded model
logged_model = 'runs:/0f8cb04450ab4b2b934a05ef8e0e26d9/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(X_val)

array([16.757378 ,  7.2064695, 20.273825 , ..., 13.494939 ,  7.2064695,
        8.691687 ], shape=(61921,), dtype=float32)

In [23]:
# or can do as xgboost model object
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

In [ ]:
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# from sklearn.svm import LinearSVR

# with open("models/preprocessor.b", "wb") as f_out:
#     pickle.dump(dv, f_out)

# mlflow.sklearn.autolog()

# for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

#     with mlflow.start_run():

#         mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#         mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
#         mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#         mlmodel = model_class()
#         mlmodel.fit(X_train, y_train)

#         y_pred = mlmodel.predict(X_val)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)
        